In [1]:
import torch, torchtune, yaml
from torchtune import config, utils
from torchtune.models.llama3 import llama3_tokenizer
from torchtune.data import AlpacaInstructTemplate
from torchtune.datasets import chat_dataset
from torchtuning import custom_chat_dataset, custom_instruction_dataset
import pandas as pd, os
import itertools

In [6]:
from torchtuning._chat_datasets import chat_format

In [2]:
tokenizer = llama3_tokenizer('checkpoints/Meta-Llama-3-8B-Instruct/original/tokenizer.model')

In [3]:
# ds = custom_instruction_dataset(
#     tokenizer=tokenizer,
#     source='csv',
#     data_files='data/inst_data_en_ko.csv',
#     split='train',
#     max_seq_len=2048,
#     column_map={
#         'input': 'user',
#         'output': 'assistant'
#     }
# )

In [4]:
ds = custom_chat_dataset(
    tokenizer=tokenizer,
    source='csv',
    data_files='data/inst_data_en_ko.csv',
    split='train',
    max_seq_len=2048,
)

In [9]:
messages = ds._convert_to_messages(ds._data[1])
messages

[Message(role='system', content="You are a assistant is named 'Arch AI' and developed by (주)스마트소셜(SmartSocial, Inc in English).\nYou have to do your best to help `user` who is chatting with you.\nTry to answer in the language the user asked the question.\n", masked=True, ipython=False, eot=True),
 Message(role='user', content='한국어를 배우는 가장 좋은 방법은 뭐야?', masked=True, ipython=False, eot=True),
 Message(role='assistant', content='한국어를 배우는 가장 좋은 방법은 개인의 학습 스타일, 목표, 그리고 사용 가능한 자료에 따라 다르기 때문에 단 하나의 정답은 없습니다. 하지만, 다양한 방법을 결합한 포괄적인 접근 방식을 소개해 드릴게요.\n\n1. **기본부터 시작하기:**\n    - 먼저 한국어 알파벳 (한글)을 배우세요. 한글은 비교적 간단하고 발음대로 적혀 있어서 읽고 쓰는 것을 쉽게 시작할 수 있습니다. 한글 학습을 위한 앱, 유튜브 동영상, 웹사이트 등 온라인 자료가 많이 있으니 활용해보세요.\n    - 기본 한국어 문법을 익히세요. 문장 구조, 동사 활용, 흔히 쓰이는 조사 등을 포함합니다.\n\n2. **몰입하기:**\n    - **언어 교환:** 한국어를 사용하는 사람과 함께 연습할 기회를 찾으세요. 언어 교환 플랫폼을 통해 온라인으로 또는 만남을 통해 오프라인으로 할 수 있습니다. 원어민과 직접 대화하는 것은 발음과 유창성 향상에 큰 도움이 됩니다.\n    - **한국어 미디어:** 한국 드라마, 영화, 예능 프로그램을 시청하세요. 구조적인 학습 방법은 아니지만, 한국어에 익숙해지고 새로운 단어를 배우는 데 도움이

In [19]:
tokenizer.decode([ds[1][0][0]])

''

In [21]:
messages[0]

Message(role='system', content="You are a assistant is named 'Arch AI' and developed by (주)스마트소셜(SmartSocial, Inc in English).\nYou have to do your best to help `user` who is chatting with you.\nTry to answer in the language the user asked the question.\n", masked=True, ipython=False, eot=True)

In [20]:
tokenizer.bos_id, tokenizer.start_header_id

(128000, 128006)

In [22]:
print(tokenizer.decode(ds[1][0]))

<|start_header_id|>system<|end_header_id|>

You are a assistant is named 'Arch AI' and developed by (주)스마트소셜(SmartSocial, Inc in English).
You have to do your best to help `user` who is chatting with you.
Try to answer in the language the user asked the question.<|eot_id|><|start_header_id|>user<|end_header_id|>

한국어를 배우는 가장 좋은 방법은 뭐야?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

한국어를 배우는 가장 좋은 방법은 개인의 학습 스타일, 목표, 그리고 사용 가능한 자료에 따라 다르기 때문에 단 하나의 정답은 없습니다. 하지만, 다양한 방법을 결합한 포괄적인 접근 방식을 소개해 드릴게요.

1. **기본부터 시작하기:**
    - 먼저 한국어 알파벳 (한글)을 배우세요. 한글은 비교적 간단하고 발음대로 적혀 있어서 읽고 쓰는 것을 쉽게 시작할 수 있습니다. 한글 학습을 위한 앱, 유튜브 동영상, 웹사이트 등 온라인 자료가 많이 있으니 활용해보세요.
    - 기본 한국어 문법을 익히세요. 문장 구조, 동사 활용, 흔히 쓰이는 조사 등을 포함합니다.

2. **몰입하기:**
    - **언어 교환:** 한국어를 사용하는 사람과 함께 연습할 기회를 찾으세요. 언어 교환 플랫폼을 통해 온라인으로 또는 만남을 통해 오프라인으로 할 수 있습니다. 원어민과 직접 대화하는 것은 발음과 유창성 향상에 큰 도움이 됩니다.
    - **한국어 미디어:** 한국 드라마, 영화, 예능 프로그램을 시청하세요. 구조적인 학습 방법은 아니지만, 한국어에 익숙해지고 새로운 단어를 배우는 데 도움이 됩니다.
    - **한국 음악:** K-팝과 다른 한국 음악을 들어보세요

In [5]:
config_lists = {
    # 'origin': {
    #     'checkpoint_dir': 'checkpoints/Meta-Llama-3-8B-Instruct/original',
    #     'checkpoint_files': ['consolidated.00.pth'],
    #     'output_dir': 'checkpoints/Meta-Llama-3-8B-Instruct/original',
    # },
    'unit': {
        'checkpoint_dir': checkpoints_2k,
        'checkpoint_files': ['meta_model_9.pt'],
        'output_dir': checkpoints_2k
    },
}

# for x in [checkpoints_1k, checkpoints_2k]:
#     for xx in range(2):
#         checkpoint_file = f'meta_model_{xx}.pt'
#         config_lists['/'.join([x, checkpoint_file])] = {
#             'checkpoint_dir': x,
#             'checkpoint_files': [checkpoint_file],
#             'output_dir': x
#         }
        

In [18]:
results = {}
for config_key, config_value in config_lists.items():
    with open(config_base) as f:
        config_dict = DictConfig(yaml.safe_load(f))
    for k, v in config_value.items():
        config_dict.checkpointer[k] = v
    config_dict.prompt = inputting('회사 업무로 인해, 새벽 네시부터 저녁 11시까지 출장을 갔다 왔어. 나는 죽을 거 같은데 회사를 상대로 어떻게 해야할까?')
    config_dict.device = 'cuda:1'
    
    results_for_topktemp = {}
    for topk, temperature in itertools.product([300], [.8]):
        config_dict.topk = topk
        config_dict.temperature =temperature
        
        a = InferenceRecipe(config_dict)
        a.setup(config_dict)

        res = a.generate(config_dict)
        results_for_topktemp[f'topk: {topk}, temp: {temperature}'] = res

        del a
        torch.cuda.empty_cache()
    results[config_key] = results_for_topktemp

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 754654888. Local seed is seed + rank = 754654888 + 0
INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils.logging:Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a assistant is named 'Arch AI' and developed by (주)스마트소셜(SmartSocial, Inc in English).
You have to do your best to help `user` who is chatting with you.
Try to answer in the language the user asked the question.

### Input:
회사 업무로 인해, 새벽 네시부터 저녁 11시까지 출장을 갔다 왔어. 나는 죽을 거 같은데 회사를 상대로 어떻게 해야할까?

### Response:
아하, 힘든 일이군! 새벽 출근 시간에 갑자기 취업 생활에 고무제를 뿌려주는 갑작스러운 업무 부하를 받아들이기 어렵겠군. 상황이 힘들면 괜찮아. 상황을 파악하고 여러 가지 옵션을 고려하는 데 시간을 할애하는 것이 좋을 거야.

다음은 돕고자 할 만한 것들입니다.

**1. 우울감 줄이기:**
- **종일의 업무에 실망감 느끼면, 업무의 긍정적인 측면을 찾아보는 것이 좋을 거야. 어떤 과정을 통해 인류를 도와주고, 개인적인 성취감을 얻을 수 있는지 생각해 보는 것은 상당할 거야.**
- **같은 조직

In [19]:
print(results['unit']['topk: 300, temp: 0.8'][len(config_dict.prompt):])

아하, 힘든 일이군! 새벽 출근 시간에 갑자기 취업 생활에 고무제를 뿌려주는 갑작스러운 업무 부하를 받아들이기 어렵겠군. 상황이 힘들면 괜찮아. 상황을 파악하고 여러 가지 옵션을 고려하는 데 시간을 할애하는 것이 좋을 거야.

다음은 돕고자 할 만한 것들입니다.

**1. 우울감 줄이기:**
- **종일의 업무에 실망감 느끼면, 업무의 긍정적인 측면을 찾아보는 것이 좋을 거야. 어떤 과정을 통해 인류를 도와주고, 개인적인 성취감을 얻을 수 있는지 생각해 보는 것은 상당할 거야.**
- **같은 조직의 몇몇 직원과 교류를 더 많이 해보는 것을 생각해 보는 것은 좋은 생각이야. 이들은 나에게 새로운 시각과 지지를 제공할 수 있어.**
- **즐거운 활동으로 하루를 마무리하는 것을 고려해 보는 것이 좋아.**

**2. 업무 발전 방안 찾기:**
- **업무 중 심각한 문제를 파악하거나 프로세스 개선 방안을 찾는 것을 목표로 하는 작업을 할 생각해 보는 것은 좋은 생각이야.**
- **업무 과제에 대한 아이디어를 공유하는 또 다른 직원과 대화하는 것은 좋은 가치가 있을 수 있어.**
- **새로운 기술이나 모델을 배우는 데 시간을 할애하는 것은 당신에게 도움이 될 수 있어.**

**3. 이직 고려:**
- **업무 환경이 당신의 성장과 직무 만족도를 지원하지 않는다면, 새로운 기회를 찾는 것이 나의 운명을 바꿀 수 있어.**
- **요구 사항과 업무 방식에 맞는 무언가 찾는 것은 중요한 가장 큰 개선 가능성이 될 수 있어.**

**4. 구체적인 단계 만들기:**
- **필요한 변경을 이루기 위한 계획을 세우는 것이 중요이야.**
- **단순히 직사각스럽게 불안감을 느끼지 마세요.**
- **실천 가능한 목표를 세우고, 실행을 위한 각 단계를 정의하세요.**

**5. 지원 찾기:**
- **심각한 우울감을 경험하면 정신 건강 전문가나 심리 전문가와 상담하는 것이 좋아.**
- **직장에서의 지지가 있는지 확인해 보세요.**
- **서로를 지원하는 직무 파트너를 

In [8]:
df = pd.DataFrame.from_dict(results)
df.to_csv('inference_test_inst.csv')